In [1]:
import pickle
import numpy
import sys
from music21 import instrument, note, stream, chord, duration
from utils import create_network, prepare_sequences

/Users/davidfoster/.virtualenvs/gdl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## load the lookup tables

In [2]:
with open('data/distincts', 'rb') as filepath:
    distincts = pickle.load(filepath)
    note_names, n_notes, duration_names, n_durations = distincts

with open('data/lookups', 'rb') as filepath:
    lookups = pickle.load(filepath)
    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups

## build the model

In [23]:
model = create_network(n_notes, n_durations)

# Load the weights to each node
weight_source = './weights/weights.h5' 
model.load_weights(weight_source)

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    185200      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    1800        input_4[0][0]                    
__________________________________________________________________________________________________
concatenat

## build your own phrase

In [25]:
notes = ['START', 'D3', 'D3', 'E3', 'D3', 'G3', 'F#3']
durations = [0, 0.75, 0.25, 1, 1, 1, 2]

# notes = ['START']
# durations = [0]

sequence_length = len(notes)

## Generate notes from the neural network based on a sequence of notes

In [47]:
temperature=0.3

def sample_with_temp(preds, temperature):
    preds = numpy.log(preds) / temperature
    exp_preds = numpy.exp(preds)
    preds = exp_preds / numpy.sum(exp_preds)
    return numpy.random.choice(len(preds), p=preds)

prediction_output = []
notes_pattern = [note_to_int[char] for char in notes]
durations_pattern = [duration_to_int[char] for char in durations]

for i1, i2 in zip(notes_pattern,durations_pattern):
    note_result = int_to_note[i1]
    duration_result = int_to_duration[i2]
    prediction_output.append([note_result, duration_result])
    

max_extra_notes = 200
max_seq_len = 32
for note_index in range(max_extra_notes):

    prediction_input = [numpy.array([notes_pattern])
                        , numpy.array([durations_pattern])
                       ]

    notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    
    i1 = sample_with_temp(notes_prediction[0], temperature)
    i2 = sample_with_temp(durations_prediction[0], temperature)
    

    note_result = int_to_note[i1]
    duration_result = int_to_duration[i2]
    
    prediction_output.append([note_result, duration_result])

    notes_pattern.append(i1)
    durations_pattern.append(i2)
    
    if len(notes_pattern) > max_seq_len:
        notes_pattern = notes_pattern[1:]
        durations_pattern = durations_pattern[1:]
        
    print(note_result)
        
    if note_result == 'END':
        break
        

G3
A3
B-3
D3
B-2
D3
B-2
B-3
A3
A3
G3
F3
E3
F3
C3
F2
E-3
G3
G3
B-3
F3
G3
G#3
G3
G3
F3
E3
D3
C3
B-3
E3
C4
C#4
C4
B-3
A3
B-3
E-4
D4
F4
D4
E-4
F4
C#4
C4
B-3
E-3.C4
C#4
E-4
C4
C#4
B-3
C4
B-3
G#3
G3
F3
E3
F3
G3
G#3
F2
B-3
G#3
G3
G#3
C3
D3
E-3
F3
B-2
G#3
G3
G#3
G3
C3
B-2
B-3
C4
C#4
E-4
G3
G3
G#3
G3
F3
E-3
D3
E-3
G3
G#3
G3
G#3
F3
B-3
G#3
G3
F3
G#3
D3
E-3
B-2
C3
D3
G#3
B-3
C4
C#4
G3
F3
E-3
G3
D3
B-3
G#3
G3
F3
E-3
D3
C3
B-2
C3
D3
E-3
F3
G3
G#3
G3
F3
E-3
D3
E-3
C3
A2
B-2
C3
D3
E-3
F3
G3
E-3
D3
C3
E-3
B-2
C3
D3
E-3
F3
G3
G#3
G3
F3
E-3
D3
E-3
D3
B-2
C3
D3
E-3
F3
G3
G#3
B-3
C4
B-3
C4
C#4
C4
B-3
G#3
G3
F3
E-3
C#3
C3
C#3
B-2
G#2
F3
G3
G#3
B-3
C4
C#4
C4
C#4
C4
B-3
G#3
G3
F3
E3
F3
G3
G#3
B-3
C4
C#4


## convert the output from the prediction to notes and create a midi file from the notes 

In [48]:
midi_stream = stream.Stream()

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    note_pattern, duration_pattern = pattern
    # pattern is a chord
    if ('.' in note_pattern):
        notes_in_chord = note_pattern.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    elif note_pattern == 'rest':
    # pattern is a rest
        new_note = note.Rest()
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Violoncello()
        midi_stream.append(new_note)
    elif note_pattern not in ('START', 'END'):
    # pattern is a note
        new_note = note.Note(note_pattern)
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Violoncello()
        midi_stream.append(new_note)



midi_stream = midi_stream.chordify()

midi_stream.write('midi', fp='./output/output.mid')

'test_output.mid'